In [13]:
from musical_festival_lineup.musical_festival_lineup import MusicalFestivalSolution
from musical_festival_lineup.musical_festival_data import MusicalFestivalData
from musical_festival_lineup.utils import visualize_musical_lineup
import numpy as np
import time

In [2]:
%load_ext autoreload
%autoreload 2

### 1. Structuring the problem

The  objective  is  to  design  the  optimal  festival  lineup  by  scheduling  artists  across  stages  and 
time  slots while:
- Maximizing  prime  slot  popularity
- Ensuring  genre  diversity  among  stages 
- Minimizing  fan conflicts  at each time slot. 

__`Details`__

- **Artists**: An  artist is  characterized  by  a  popularity  score  (from  0  to  100) a genre  and  fan  base.
    <br>This information with be on __`artists`__ from musical_festival_data

- **Conflicts**:   with  other  artists  (score  from  0  to  1).  Conflicts  arise  when  artists  with  overlapping 
fan  bases  perform  at  the  same  time  but  on  different  stages,  which  can  negatively  impact 
attendance.
    <br>This information will be on __`conflits`__ from musical_festival_data


### 2. Defining a  solution/individual

Each  individual  represents  a  festival  lineup,  specifying  which  artist  plays  on  which  stage  and 
at  which  time  slot. 
These  are  the  constraints  : 
- Each artist is assigned to exactly one stage and slot 
- All time slots start and end at the same time. 
- All stages have the same amount of slots
- These artists should be distributed across  **5 stages  with  7 time slots each**

__`Important`__ 
<br>Any  lineup  where  an  artist  is  assigned  to  **multiple  time  slots  or 
left  unassigned** is  not  part  of  the  search  space  and  is  not  considered  a  solution.  It  is 
forbidden to generate such an arrangement during evolution. 


__`we propose a  having  a solution as`__:

- A string of 5 (stages)*7(slots)= 35 numbers of **1** to **35**
- As for the genetic algorithms, we should have string of the same size, we will be appending a 0 to 1 digit numbers.
- So the solution will be a string of 35 * 2 (two character)= so will have i string of 70

Here a individual of our problem:

In [15]:
data=MusicalFestivalData()


In [14]:
lineup=MusicalFestivalSolution(data=data, repr="2406083423021829103033012519173112162614150532070320000422092728131121")
",".join([ str(i) for i in lineup.repr])

'24,6,8,34,23,2,18,29,10,30,33,1,25,19,17,31,12,16,26,14,15,5,32,7,3,20,0,4,22,9,27,28,13,11,21'

In [5]:
# Python code to show a random initialized solution
lineup=MusicalFestivalSolution(data=data)
",".join([ str(i) for i in lineup.repr])

'20,22,8,1,0,27,10,32,4,34,31,23,7,29,6,17,28,5,19,12,9,2,16,26,24,30,11,25,14,13,33,21,15,18,3'

- Having that, we have to options:
- 1. The first 7 numbers represent the stage 1 , having each of the 7 representing the slots in  order (1 to 7) by the slot?
- 2. Or the first 5 would represent the slot 1 (first time slot) , having the each of the 5 representing the stages in order (1 to 5)?

For that we choose the option 2, because how our fitness will be defined. It is well explained in the session 3, when defining fitness


In [6]:
visualize_musical_lineup("2406083423021829103033012519173112162614150532070320000422092728131121", data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,24: Shadow Cadence|Jazz|66,6: Aurora Skies|Pop|75,8: Crimson Harmony|Classical|20,34: Parallel Dimension|Electronic|58,23: Electric Serpents|Electronic|99
1,Slot 2,2: Velvet Pulse|Jazz|35,18: Velvet Underground|Rock|72,29: Harmonic Dissonance|Classical|96,10: The Wandering Notes|Jazz|84,30: Turbo Vortex|Rock|53
2,Slot 3,33: Cosmic Frequency|Rock|53,1: Solar Flare|Electronic|78,25: Rhythm Alchemy|Jazz|94,19: Astral Tide|Electronic|69,17: Nightfall Sonata|Classical|84
3,Slot 4,31: The Jazz Nomads|Jazz|64,12: Blue Horizon|Pop|51,16: Mystic Rhythms|Classical|78,26: Cloud Nine Collective|Pop|97,14: Synthwave Saints|Rock|94
4,Slot 5,15: Golden Ember|Rock|61,5: Echo Chamber|Electronic|98,32: The Bassline Architects|Hip-Hop|61,7: Static Mirage|Rock|94,3: Neon Reverie|Electronic|100
5,Slot 6,20: The Sonic Drifters|Rock|88,0: Midnight Echo|Rock|75,4: The Silver Owls|Classical|85,22: Quantum Beat|Hip-Hop|96,9: Deep Resonance|Jazz|90
6,Slot 7,27: Hypnotic Echoes|Rock|77,28: The Polyrhythm Syndicate|Jazz|66,13: Lunar Spectrum|Rock|99,11: Phantom Groove|Hip-Hop|47,21: Celestial Voyage|Electronic|95


In [7]:
visualize_musical_lineup(lineup.repr, data.artists)

,Slot,Stage 1,Stage 2,Stage 3,Stage 4,Stage 5
0,Slot 1,20: The Sonic Drifters|Rock|88,22: Quantum Beat|Hip-Hop|96,8: Crimson Harmony|Classical|20,1: Solar Flare|Electronic|78,0: Midnight Echo|Rock|75
1,Slot 2,27: Hypnotic Echoes|Rock|77,10: The Wandering Notes|Jazz|84,32: The Bassline Architects|Hip-Hop|61,4: The Silver Owls|Classical|85,34: Parallel Dimension|Electronic|58
2,Slot 3,31: The Jazz Nomads|Jazz|64,23: Electric Serpents|Electronic|99,7: Static Mirage|Rock|94,29: Harmonic Dissonance|Classical|96,6: Aurora Skies|Pop|75
3,Slot 4,17: Nightfall Sonata|Classical|84,28: The Polyrhythm Syndicate|Jazz|66,5: Echo Chamber|Electronic|98,19: Astral Tide|Electronic|69,12: Blue Horizon|Pop|51
4,Slot 5,9: Deep Resonance|Jazz|90,2: Velvet Pulse|Jazz|35,16: Mystic Rhythms|Classical|78,26: Cloud Nine Collective|Pop|97,24: Shadow Cadence|Jazz|66
5,Slot 6,30: Turbo Vortex|Rock|53,11: Phantom Groove|Hip-Hop|47,25: Rhythm Alchemy|Jazz|94,14: Synthwave Saints|Rock|94,13: Lunar Spectrum|Rock|99
6,Slot 7,33: Cosmic Frequency|Rock|53,21: Celestial Voyage|Electronic|95,15: Golden Ember|Rock|61,18: Velvet Underground|Rock|72,3: Neon Reverie|Electronic|100


- Get artists of slot

In [8]:
lineup._get_slot_repr_list(slot=6)

[33, 21, 15, 18, 3]

### 3. Defining fitness

" The  quality  of  a  festival  lineup  is  determined  by  balancing  __`three  equally`__  important 
objectives** ,  each  contributing  to  an  overall  score.  Since  these  objectives  have  different 
scales,  ** they  must  be  normalized  to  the  same  range  (between  0  and  1)  to  ensure  they 
contribute equally to the final fitness score.
<br>
 These are the objectives: 
 
__`Prime  Slot  Popularity`__  :  The  most  popular  artists  should  be  scheduled  in  the  prime 
slots  (**the  last  time  slot  on  each  stage**).  This  score  is  calculated  by  normalizing  the 
total  popularity  of  artists  in  prime  slots  against  the  maximum  possible  total  popularity 
(e.g. if only most popular artists - score 100 - were scheduled on the prime slot). 

__`Genre  Diversity`__  :  A  diverse  distribution  of  **genres  among  stages  in  each  time  slot**
improves  the  festival  experience.  This  score  is  determined  by  normalizing  the 
number  of  unique  genres  in  each  slot  relative  to  the  maximum  possible  number  of 
unique  genres  (e.g.  if  only  different  genres  were  scheduled  in  that  time  slot)  .  Then 
you take the average among time slots. 

__`Conflict  Penalty`__  : ** Fan  conflicts  occur  when  artists  with  overlapping  audiences 
perform  at  the  same  time  on  different  stages**. 
 This  score  is  calculated  by  normalizing 
the  total  conflict  value  in  each  slot  against  the  worst  possible  conflict  scenario  (e.g. 
where  all  artists  with  maximum  conflict  are  scheduled  together).  Then  you  take  the 
average  among  time  slots.  Since  conflicts  negatively  impact  the  lineup,  this  is  a 
penalization  score. 
"

__`Prime  Slot  Popularity`__  

1.  Maximum  possible  total  popularity, having into account that There is 5 stages, so we eill be summing the 5(NUM STAGES) largest

In [9]:
data.max_popularity_in_prime_slot

493

2. Total  popularity  of  artists: <br>
For that, it was defined a function that receives artists Ids and returns the popularity for that specifics Ids. <br>
Lets test it!



In [10]:
#Summing popularity of all artists
data.get_sum_popularity(list(range(14)))

1031

In [11]:
#Summing popularity of [1,2,3,4]
data.get_sum_popularity([1,2,3,4])

298

3. Normalizing the popularity against the max number of popularity

In [12]:
lineup._get_popularity_normalized([1,2,3,4])

0.6044624746450304

__`Genre  Diversity`__  : 

1. Get maximum number of genres per slot, can not be greater than the total number of stages, which is 5

In [13]:
#Genre  Diversity
data.max_distinct_genre_per_slot

5

2. Number of distinct genres : <br>
For that, it was defined a function that receives artists Ids and returns distinct genres in that list of specifics Ids. <br>
Lets test it!


In [14]:
data.get_count_distinct_genres(artists_ids_list=[0,1,10,2])

3

3. Normalizing the genres against the max number of distinct genres

In [15]:
lineup._get_genre_diversity_normalized(artists_ids_list=[0,1,10])

0.6

__`Penalty Conflict`__  : 

1. Get maximum possible conflift, the  worst  possible  conflict  scenario, so for that we get the top K worst case scenerio. <br>
 What is K?
 Is the numbers os conflits in a slot, which is the combination of 5 elements when grouping in two so, we will have C5,2, tthat is equal to 10.
 So will will be choosing the top 10 worst conflits


In [16]:
data.max_worst_conflit_per_slot

np.float64(10.0)

2. The  Total  conflict  value  in  each  slot : <br>
For that, it was defined a function that receives artists Ids and sum of all conflicts in the list. <br>
Lets test it!


In [17]:
data.get_sum_conflicts(artists_ids_list=[3,1,10])

np.float64(1.8)

In [18]:
data.get_sum_conflicts(artists_ids_list=[32, 29, 25, 13, 30])

np.float64(5.75)

In [19]:
data.get_sum_conflicts(artists_ids_list=[2,10])

np.float64(0.9)

3. Normalizing the conflicts against the worst conflit scenario

In [20]:
lineup._get_conflicts_normalized(artists_ids_list=[3,1,10])

np.float64(0.18)

In [21]:
lineup._get_conflicts_normalized(artists_ids_list=[32, 29, 25, 13, 30])

np.float64(0.575)

##### 3.2 Other important functions in the solution

In [22]:
lineup.repr

[20,
 22,
 8,
 1,
 0,
 27,
 10,
 32,
 4,
 34,
 31,
 23,
 7,
 29,
 6,
 17,
 28,
 5,
 19,
 12,
 9,
 2,
 16,
 26,
 24,
 30,
 11,
 25,
 14,
 13,
 33,
 21,
 15,
 18,
 3]

- Get the list of artists Ids in a Slot, Slot is a parameter

In [23]:
lineup._get_slot_repr_list(1)

[27, 10, 32, 4, 34]

In [24]:
lineup._get_slot_repr_list(0)

[20, 22, 8, 1, 0]

##### 3.3 Fitness

In [16]:
#Random 1
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [19, 11, 3, 2, 27]
Slot 0: Conflitcs: 0.5000000000000001, genres: 0.8, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.5000000000000001)], List of genres: [0.8], Popularity of the prime slot: 0
Slot of artists: [33, 21, 30, 6, 32]
Slot 1: Conflitcs: 0.55, genres: 0.8, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.5000000000000001), np.float64(0.55)], List of genres: [0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [26, 12, 0, 28, 1]
Slot 2: Conflitcs: 0.6050000000000001, genres: 0.8, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.5000000000000001), np.float64(0.55), np.float64(0.6050000000000001)], List of genres: [0.8, 0.8, 0.8], Popularity of the prime slot: 0
Slot of artists: [7, 16, 17, 9, 24]
Slot 3: Conflitcs: 0.595, genres: 0.6, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.5000000000000001), np.float64(0.55), np.float64(0.6050000000000001), np.float64(0.595)], List of genres: [0.8, 0.8, 0.8, 0.6],

np.float64(0.8806215589684149)

In [17]:
#Random 2
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [14, 25, 1, 12, 21]
Slot 0: Conflitcs: 0.33999999999999997, genres: 0.8, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.33999999999999997)], List of genres: [0.8], Popularity of the prime slot: 0
Slot of artists: [13, 29, 17, 0, 30]
Slot 1: Conflitcs: 0.575, genres: 0.4, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.33999999999999997), np.float64(0.575)], List of genres: [0.8, 0.4], Popularity of the prime slot: 0
Slot of artists: [20, 9, 4, 5, 8]
Slot 2: Conflitcs: 0.5399999999999999, genres: 0.8, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.33999999999999997), np.float64(0.575), np.float64(0.5399999999999999)], List of genres: [0.8, 0.4, 0.8], Popularity of the prime slot: 0
Slot of artists: [24, 22, 33, 28, 10]
Slot 3: Conflitcs: 0.5000000000000001, genres: 0.6, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.33999999999999997), np.float64(0.575), np.float64(0.5399999999999999), np.float64(0.5000000000000001)], L

np.float64(0.8734279918864097)

In [21]:
#Random 3
lineup=MusicalFestivalSolution(data=data)
lineup.repr
lineup.fitness(verbose=True)

Slot of artists: [8, 13, 15, 21, 9]
Slot 0: Conflitcs: 0.335, genres: 0.8, sum_popularity: 0
Slot 0: List of  Conflitcs: [np.float64(0.335)], List of genres: [0.8], Popularity of the prime slot: 0
Slot of artists: [16, 19, 26, 34, 1]
Slot 1: Conflitcs: 0.45999999999999996, genres: 0.6, sum_popularity: 0
Slot 1: List of  Conflitcs: [np.float64(0.335), np.float64(0.45999999999999996)], List of genres: [0.8, 0.6], Popularity of the prime slot: 0
Slot of artists: [14, 28, 18, 2, 7]
Slot 2: Conflitcs: 0.725, genres: 0.4, sum_popularity: 0
Slot 2: List of  Conflitcs: [np.float64(0.335), np.float64(0.45999999999999996), np.float64(0.725)], List of genres: [0.8, 0.6, 0.4], Popularity of the prime slot: 0
Slot of artists: [32, 20, 4, 12, 31]
Slot 3: Conflitcs: 0.48, genres: 1.0, sum_popularity: 0
Slot 3: List of  Conflitcs: [np.float64(0.335), np.float64(0.45999999999999996), np.float64(0.725), np.float64(0.48)], List of genres: [0.8, 0.6, 0.4, 1.0], Popularity of the prime slot: 0
Slot of arti

np.float64(1.229917415241959)